In [1]:
#!pip install river
#!pip install creme

In [2]:
#!pip install sklearn

In [3]:
import pandas as pd
import river
import creme
import re
from sklearn.model_selection import train_test_split
#import numpy as np

C:\Users\dedee\AppData\Roaming\Python\Python310\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
from river import metrics


In [5]:
df = pd.read_csv('cyberbullying_tweets.csv')

In [6]:
df = df.sample(frac = 1)

In [7]:
df.head()

,tweet_text,cyberbullying_type
40935,Fuck dat sad ass hoe dawg ! Lol @ThaCockyKind:...,ethnicity
12341,Really.... is that what you got out of all of ...,gender
14705,Christian jokes. Islam jokes. Abortion jokes. ...,gender
29909,@alexlifschitz YUP.,other_cyberbullying
33763,I hated them... I wanted them to be leaked. My...,age


In [8]:
df['tweet_text'] = df['tweet_text'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')

C:\Users\dedee\AppData\Local\Temp/ipykernel_239160/1079813231.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['tweet_text'] = df['tweet_text'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')


In [9]:
df.columns

Index(['tweet_text', 'cyberbullying_type'], dtype='object')

In [10]:
df.shape

(47692, 2)

In [11]:
df.groupby('cyberbullying_type').count()

,tweet_text
cyberbullying_type,
age,7992
ethnicity,7961
gender,7973
not_cyberbullying,7945
other_cyberbullying,7823
religion,7998


In [12]:
df.isnull().sum()

tweet_text            0
cyberbullying_type    0
dtype: int64

In [13]:
df1 = df[:47600]

In [14]:
df2 = df[47600:]

In [15]:
print(df1.shape)
print(df2.shape)

(47600, 2)
(92, 2)


In [16]:
df2.groupby('cyberbullying_type').count()

,tweet_text
cyberbullying_type,
age,18
ethnicity,11
gender,18
not_cyberbullying,18
other_cyberbullying,11
religion,16


In [17]:
train_ds,test_ds=train_test_split(df1)

In [18]:
print(train_ds.shape)
print(test_ds.shape)

(35700, 2)
(11900, 2)


In [19]:
from river.naive_bayes import MultinomialNB

In [20]:
from river.feature_extraction import BagOfWords,TFIDF

In [21]:
from river.compose import Pipeline

In [22]:
train_ds = train_ds.to_records(index=False)


In [23]:
train_ds

rec.array([("Scoring like a Cunt because you can't cook for shit isn't fighting hard Kat.... You stupid mole #mkr", 'gender'),
           ('GRECIAN FUCK SHOES.', 'other_cyberbullying'),
           ('That’s not necessarily a homophobic joke. It’s def annoying! But it’s more of a rape joke than a gay joke. Generally b/c the “tops” of the joke are heterosexual. But I get it! It’s dumb. But it doesn’t mean allies are any less supportive of our community as a whole. It’s a joke.', 'gender'),
           ...,
           ('Racism is wen u select your white clothes to wash before the colored ones... Say no to Racism!!! Wash them all together :D', 'ethnicity'),
           ('Also Rihanna is my fave. Shes that popular girl in school who’s actually nice &amp; sticks up to the bully who thinks their popular cause they continuously put other pple down to look “cool” in front of their friends.', 'age'),
           ('Ramzan preaches spreading anti hindu hatred? Ever uttered a word on islamic terrorism?

In [24]:
pline = Pipeline(('vectorizer',TFIDF()),('nb',MultinomialNB()))

In [25]:
pline

Pipeline (
  TFIDF (
    normalize=True
    on=None
    strip_accents=True
    lowercase=True
    preprocessor=None
    tokenizer=None
    ngram_range=(1, 1)
  ),
  MultinomialNB (
    alpha=1.
  )
)

In [26]:
pline.steps

OrderedDict([('vectorizer',
              TFIDF (
                normalize=True
                on=None
                strip_accents=True
                lowercase=True
                preprocessor=None
                tokenizer=None
                ngram_range=(1, 1)
              )),
             ('nb',
              MultinomialNB (
                alpha=1.
              ))])

In [27]:
for text,label in train_ds:
    pline = pline.learn_one(text,label)

In [28]:
pline.predict_proba_one("do not make sucide jokes")

{'gender': 0.8288199594156551,
 'other_cyberbullying': 0.03948328901890589,
 'religion': 0.04105196262135837,
 'ethnicity': 0.030515837163547548,
 'age': 0.026000046279746426,
 'not_cyberbullying': 0.034128905500786386}

In [29]:
test_ds.iloc[:2]

,tweet_text,cyberbullying_type
25982,What the shit.,other_cyberbullying
1031,ACHEI UM TEMA o/ bullying contra casais homoss...,not_cyberbullying


In [30]:
test_ds = test_ds.to_records(index = False)

In [31]:
metric = river.metrics.Accuracy()

In [32]:
for text,label in test_ds:
    y_pred = pline.predict_one(text)
    metric = metric.update(label,y_pred)
    pline = pline.learn_one(text,label)

In [33]:
metric

Accuracy: 76.23%